In [ ]:
import time
import random
import sklearn
import sklearn.datasets
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from cvxpy import *
from scipy import stats
from scipy import sparse
from scipy import linalg
from keras.datasets import mnist
from scipy.linalg import eigh
from scipy.sparse import linalg
from scipy.sparse import csr_array
from scipy.sparse import coo_array
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import eigs, eigsh
from sklearn.preprocessing import normalize
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.pairwise import euclidean_distances

In [ ]:
def sigma_mat(dist, k):
  dist.sort(axis=1)
  dist_vec = np.array([dist[:, k-1]])
  sigma_mat = 2 * np.outer(dist_vec, dist_vec)
  return sigma_mat


def DM_coordinates(v1, v2, k):
  distsI = euclidean_distances(v1, v1)
  dist_matI = np.copy(distsI)
  sigma_matI = sigma_mat(dist_matI, k)

  KI = np.exp(-distsI**2 / sigma_matI)
  KI = KI - np.diag(np.diag(KI))
  DI = np.diag(np.sum(KI, axis=1))
  D_leftI = np.diag(np.sum(KI,axis = 1)**-0.5)
  D_rightI = np.diag(np.sum(KI,axis = 1)**0.5)
  rand_walk_LI = np.matmul(np.linalg.inv(DI), KI)
  RWL_primeI = np.matmul(D_rightI, np.matmul(rand_walk_LI, D_leftI))

  eigenValuesI, eigenVectorsI = eigh(RWL_primeI)
  idxI = eigenValuesI.argsort()[::-1]
  d_lamI = np.diag(eigenValuesI[idxI])
  eig_I = D_leftI@eigenVectorsI[:, idxI]

  diffusion_coordinatesI = np.matmul(eig_I, d_lamI)
  diffusion_coordinatesI_df = pd.DataFrame(diffusion_coordinatesI)

  distsII = euclidean_distances(v2, v2)
  dist_matII = np.copy(distsII)
  sigma_matII = sigma_mat(dist_matII, k)

  KII = np.exp(-distsII**2 / sigma_matII)
  KII = KII - np.diag(np.diag(KII))
  DII = np.diag(np.sum(KII, axis=1))
  D_leftII = np.diag(np.sum(KII,axis = 1)**-0.5)
  D_rightII = np.diag(np.sum(KII,axis = 1)**0.5)
  rand_walk_LII = np.matmul(np.linalg.inv(DII), KII)
  RWL_primeII = np.matmul(D_rightII, np.matmul(rand_walk_LII, D_leftII))

  eigenValuesII, eigenVectorsII = eigh(RWL_primeII)
  idxII = eigenValuesII.argsort()[::-1]
  d_lamII = np.diag(eigenValuesII[idxII])
  eig_II = D_leftII@eigenVectorsII[:, idxII]

  diffusion_coordinatesII = np.matmul(eig_II, d_lamII)
  diffusion_coordinatesII_df = pd.DataFrame(diffusion_coordinatesII)
  return diffusion_coordinatesI_df, diffusion_coordinatesII_df


def check_conflicts(n, eigenvec):
  if eigenvec.shape[1] == 1:
    x_star = eigenvec
  else:
    d = eigenvec.shape[1]
    x_star = eigenvec[:, (d - 1)]

  list_x = range(1, n+1)
  list_y = range(1, n+1)

  pairs = []
  for x in list_x:
    for y in list_y:
      pairs.append((x, y))

  df = pd.DataFrame(pairs)
  df["x_star"] = x_star
  df.columns = ["row_idx", "col_idx", "x_star"]
  df_new = df

  first_point = []
  second_point = []

  for i in range(n):
    idx = np.argmax(df_new.x_star)
    r_idx = df_new.iloc[idx, 0]
    first_point.append(r_idx)
    c_idx = df_new.iloc[idx, 1]
    second_point.append(c_idx)
    df_new = df_new[df_new.row_idx != r_idx]
    df_new = df_new[df_new.col_idx != c_idx]

  one_to_one = pd.DataFrame(first_point)
  one_to_one["second_point"] = second_point
  one_to_one.columns = ["first_point", "second_point"]
  return one_to_one


def ARI_rate(pairs, n1):
  idx1 = pairs.iloc[:,0]
  vec = idx1 - 1
  vec1 = np.where(vec <= n1-1, 0, vec)
  vec1 = np.where(vec1 >= n1, 1, vec1)

  idx2 = pairs.iloc[:,1]
  vec = idx2 - 1
  vec2 = np.where(vec <= n1-1, 0, vec)
  vec2 = np.where(vec2 >= n1, 1, vec2)

  return adjusted_rand_score(vec1, vec2)



def kronecker_product2(A, B, n):
  a_w = np.where(A != 0)
  b_w = np.where(B != 0)
  r_a = list(a_w[0])
  c_a = list(a_w[1])
  r_b = list(b_w[0])
  c_b = list(b_w[1])

  R1 = (np.outer(r_a, np.ones((1, len(r_b)))) * n).flatten() + (np.outer(np.ones((len(r_a), 1)), r_b)).flatten()
  C1 = (np.outer(c_a, np.ones((1, len(c_b)))) * n).flatten() + (np.outer(np.ones((len(c_a), 1)), c_b)).flatten()
  V1 = (np.outer(A[r_a, c_a].flatten(), np.ones((1, len(r_b))))) * (np.outer(np.ones((len(c_a), 1)), B[r_b, c_b].flatten()))
  res = coo_array((V1.flatten(), (R1, C1)), shape = (n*n, n*n))
  return res


def compute_M2(X, Y, k):
  x_dist = euclidean_distances(X,X)
  idx_x = np.argpartition(x_dist, range(k))[:, k:]
  x_dist[np.arange(idx_x.shape[0])[:,None], idx_x] = 2
  x_dist = 2 - x_dist
  x_dist = 0.5*(x_dist+x_dist.T)

  y_dist = euclidean_distances(Y,Y)
  idx_y = np.argpartition(y_dist, range(k))[:, k:]
  y_dist[np.arange(idx_y.shape[0])[:,None], idx_y] = 2
  y_dist = 2 - y_dist
  y_dist = 0.5*(y_dist+y_dist.T)

  n = x_dist.shape[0]
  M2 = kronecker_product2(x_dist, y_dist, n)
  return M2

In [ ]:
def compute_M2_new(X, Y, k):
  x_dist = euclidean_distances(X,X)
  idx_x = np.argpartition(x_dist, range(k))[:, k:]
  x_dist[np.arange(idx_x.shape[0])[:,None], idx_x] = 2
  x_dist = 2 - x_dist
  Q_x = 0.5*(x_dist+x_dist.T)

  y_dist = euclidean_distances(Y,Y)
  idx_y = np.argpartition(y_dist, range(k))[:, k:]
  y_dist[np.arange(idx_y.shape[0])[:,None], idx_y] = 2
  y_dist = 2 - y_dist
  Q_y = 0.5*(y_dist+y_dist.T)

  return Q_x, Q_y

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(60000, 784)
y_train = y_train.reshape(60000, 1)

idx = np.random.choice(X_train.shape[0], 50000, replace=False)
mnist_data = X_train[idx]
mnist_data = pd.DataFrame(mnist_data)

mnist_lab = y_train[idx]
mnist_lab = pd.DataFrame(mnist_lab)
mnist_lab.columns = ['lable']

dat = pd.concat([mnist_data, mnist_lab], axis=1)


dat0 = dat.loc[dat['lable'] == 0]
dat1 = dat.loc[dat['lable'] == 1]
dat2 = dat.loc[dat['lable'] == 2]
dat3 = dat.loc[dat['lable'] == 3]
dat4 = dat.loc[dat['lable'] == 4]
dat5 = dat.loc[dat['lable'] == 5]
dat6 = dat.loc[dat['lable'] == 6]
dat7 = dat.loc[dat['lable'] == 7]
dat8 = dat.loc[dat['lable'] == 8]
dat9 = dat.loc[dat['lable'] == 9]


def data_generator_MNIST(n, datA, datB):
  p_mnist = 784

  v1 = datA.sample(n = n)
  v2 = datB.sample(n = n)
  datA_ = pd.concat([v1.iloc[:(int(n/2)),:], v2.iloc[:(int(n/2)),:]])
  datB_ = pd.concat([v1.iloc[(int(n/2)):,:], v2.iloc[(int(n/2)):,:]])

  X1_mnist = datA_.iloc[:, 0:p_mnist]
  X2_mnist = datB_.iloc[:, 0:p_mnist]
  Y1_mnist = datA_.iloc[:, p_mnist]
  Y2_mnist = datB_.iloc[:, p_mnist]
  return X1_mnist, X2_mnist, Y1_mnist, Y2_mnist

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
def matching_rate_M2_mixed_CC(n, k, num_coords, M, data_func, *args):
  st = time.time()
  n1 = int(n/2)

  num_ev = []
  ARI_lst = []
  ARI_lst1 = []
  ARI_lst2 = []
  ARI_lst3 = []
  ARI_lst4 = []
  ARI_lst5 = []
  ARI_lst6 = []
  ARI_lst7 = []
  ARI_lst8 = []
  ARI_lst9 = []

  for i in range(M):
    print(i)
    view_1, view_2, lab1, lab2 = data_func(*args)

    diffusion_coordinatesI_df, diffusion_coordinatesII_df = DM_coordinates(view_1, view_2, k)

    leadEV1 = diffusion_coordinatesI_df.iloc[:,1:num_coords]
    leadEV2 = diffusion_coordinatesII_df.iloc[:,1:num_coords]

    norm_view1 = normalize(leadEV1, axis=1, norm='l2')
    norm_view2 = normalize(leadEV2, axis=1, norm='l2')

    Q_1, Q_2 = compute_M2_new(norm_view1, norm_view2, k)
    e_val_Q1, e_vec_Q1 = eigsh(Q_1, k=3)
    e_val_Q2, e_vec_Q2 = eigsh(Q_2, k=3)

    V = []
    for i in range(3):
      for j in range(3):
        V.append(np.reshape(np.outer(e_vec_Q1[:,i],e_vec_Q2[:,j]),(n**2)))

    eVector1 = V[8]
    eVector2 = V[7]
    eVector3 = V[6]
    eVector4 = V[5]
    eVector5 = V[4]
    eVector6 = V[3]
    eVector7 = V[2]
    eVector8 = V[1]
    eVector9 = V[0]

    pairs1 = check_conflicts(n, eVector1.reshape(n*n, 1))
    pairs2 = check_conflicts(n, eVector2.reshape(n*n, 1))
    pairs3 = check_conflicts(n, eVector3.reshape(n*n, 1))
    pairs4 = check_conflicts(n, eVector4.reshape(n*n, 1))
    pairs5 = check_conflicts(n, eVector5.reshape(n*n, 1))
    pairs6 = check_conflicts(n, eVector6.reshape(n*n, 1))
    pairs7 = check_conflicts(n, eVector7.reshape(n*n, 1))
    pairs8 = check_conflicts(n, eVector8.reshape(n*n, 1))
    pairs9 = check_conflicts(n, eVector9.reshape(n*n, 1))

    a1 = ARI_rate(pairs1, n1)
    ARI_lst1.append(a1)
    a2 = ARI_rate(pairs2, n1)
    ARI_lst2.append(a2)
    a3 = ARI_rate(pairs3, n1)
    ARI_lst3.append(a3)
    a4 = ARI_rate(pairs4, n1)
    ARI_lst4.append(a4)
    a5 = ARI_rate(pairs5, n1)
    ARI_lst5.append(a5)
    a6 = ARI_rate(pairs6, n1)
    ARI_lst6.append(a6)
    a7 = ARI_rate(pairs7, n1)
    ARI_lst7.append(a7)
    a8 = ARI_rate(pairs8, n1)
    ARI_lst8.append(a8)
    a9 = ARI_rate(pairs9, n1)
    ARI_lst9.append(a9)

    v1 = np.zeros((n*n,1))
    n_pairs1 = pairs1.sort_values('first_point')
    for i in range(n):
      d1 = n_pairs1.iloc[i,1]
      idx1 = (n*i) + d1-1
      v1[idx1] = 1
    P_v1 = v1.reshape(n, n)
    result_matrix1 = Q_2 @ P_v1 @ Q_1.T @ P_v1.T
    obj_res1 = np.trace(result_matrix1)

    v2 = np.zeros((n*n,1))
    n_pairs2 = pairs2.sort_values('first_point')
    for i in range(n):
      d2 = n_pairs2.iloc[i,1]
      idx2 = (n*i) + d2-1
      v2[idx2] = 1
    P_v2 = v2.reshape(n, n)
    result_matrix2 = Q_2 @ P_v2 @ Q_1.T @ P_v2.T
    obj_res2 = np.trace(result_matrix2)

    v3 = np.zeros((n*n,1))
    n_pairs3 = pairs3.sort_values('first_point')
    for i in range(n):
      d3 = n_pairs3.iloc[i,1]
      idx3 = (n*i) + d3-1
      v3[idx3] = 1
    P_v3 = v3.reshape(n, n)
    result_matrix3 = Q_2 @ P_v3 @ Q_1.T @ P_v3.T
    obj_res3 = np.trace(result_matrix3)

    v4 = np.zeros((n*n,1))
    n_pairs4 = pairs4.sort_values('first_point')
    for i in range(n):
      d4 = n_pairs4.iloc[i,1]
      idx4 = (n*i) + d4-1
      v4[idx4] = 1
    P_v4 = v4.reshape(n, n)
    result_matrix4 = Q_2 @ P_v4 @ Q_1.T @ P_v4.T
    obj_res4 = np.trace(result_matrix4)

    v5 = np.zeros((n*n,1))
    n_pairs5 = pairs5.sort_values('first_point')
    for i in range(n):
      d5 = n_pairs5.iloc[i,1]
      idx5 = (n*i) + d5-1
      v5[idx5] = 1
    P_v5 = v5.reshape(n, n)
    result_matrix5 = Q_2 @ P_v5 @ Q_1.T @ P_v5.T
    obj_res5 = np.trace(result_matrix5)

    v6 = np.zeros((n*n,1))
    n_pairs6 = pairs6.sort_values('first_point')
    for i in range(n):
      d6 = n_pairs6.iloc[i,1]
      idx6 = (n*i) + d6-1
      v6[idx6] = 1
    P_v6 = v6.reshape(n, n)
    result_matrix6 = Q_2 @ P_v6 @ Q_1.T @ P_v6.T
    obj_res6 = np.trace(result_matrix6)

    v7 = np.zeros((n*n,1))
    n_pairs7 = pairs7.sort_values('first_point')
    for i in range(n):
      d7 = n_pairs7.iloc[i,1]
      idx7 = (n*i) + d7-1
      v7[idx7] = 1
    P_v7 = v7.reshape(n, n)
    result_matrix7 = Q_2 @ P_v7 @ Q_1.T @ P_v7.T
    obj_res7 = np.trace(result_matrix7)

    v8 = np.zeros((n*n,1))
    n_pairs8 = pairs8.sort_values('first_point')
    for i in range(n):
      d8 = n_pairs8.iloc[i,1]
      idx8 = (n*i) + d8-1
      v8[idx8] = 1
    P_v8 = v8.reshape(n, n)
    result_matrix8 = Q_2 @ P_v8 @ Q_1.T @ P_v8.T
    obj_res8 = np.trace(result_matrix8)

    v9 = np.zeros((n*n,1))
    n_pairs9 = pairs9.sort_values('first_point')
    for i in range(n):
      d9 = n_pairs9.iloc[i,1]
      idx9 = (n*i) + d9-1
      v9[idx9] = 1
    P_v9 = v9.reshape(n, n)
    result_matrix9 = Q_2 @ P_v9 @ Q_1.T @ P_v9.T
    obj_res9 = np.trace(result_matrix9)

    obj_arr = np.array([float(obj_res1), float(obj_res2), float(obj_res3), float(obj_res4), float(obj_res5), float(obj_res6), float(obj_res7), float(obj_res8), float(obj_res9)])
    idx = np.argmax(obj_arr)
    max = np.max(obj_arr)
    print("The result - ev number " + str((idx+1)) + ", objective function = " + str(round(max, 3)))

    a_arr = np.array([float(a1), float(a2), float(a3), float(a4), float(a5), float(a6), float(a7), float(a8), float(a9)])
    ARI_lst.append(a_arr[idx])
    num_ev.append(idx+1)

  et = time.time()

  total_time = et - st
  print("Execution time of matching_rate: ", total_time, " seconds")

  return pairs1, pairs2, pairs3, pairs4, pairs5, pairs6, pairs7, pairs8, pairs9, view_1, view_2, ARI_lst1, ARI_lst2, ARI_lst3, ARI_lst4, ARI_lst5, ARI_lst6, ARI_lst7, ARI_lst8, ARI_lst9, num_ev, ARI_lst

In [ ]:
n_lst = [50, 60, 70, 80, 90, 100, 120, 150]
l = len(n_lst)
num_coords = 2

st = time.time()

def GM_sys_M2_mixed_CC(ns, M, num_coords):
  num_nodes = []
  run_time = []
  num_evs = []
  ARI_lsts = []

  median_ARI_M = []
  median_ARI_M_1 = []
  median_ARI_M_2 = []
  median_ARI_M_3 = []

  mean_ARI_M = []
  mean_ARI_M_1 = []
  mean_ARI_M_2 = []
  mean_ARI_M_3 = []

  std_ARI_M = []
  std_ARI_M_1 = []
  std_ARI_M_2 = []
  std_ARI_M_3 = []

  for i in range(l):
    print(i)
    num_nodes.append(ns[i])
    st = time.time()
    k_neigh = 20

    pairs1, pairs2, pairs3, pairs4, pairs5, pairs6, pairs7, pairs8, pairs9, view_1, view_2, ARI_lst1, ARI_lst2, ARI_lst3, ARI_lst4, ARI_lst5, ARI_lst6, ARI_lst7, ARI_lst8, ARI_lst9, num_ev, ARI_lst = matching_rate_M2_mixed_CC(ns[i], k_neigh, num_coords, M, data_generator_MNIST, ns[i], dat0, dat1)
    et = time.time()
    total_time = et - st
    run_time.append(total_time)
    num_evs.append(num_ev)
    ARI_lsts.append(ARI_lst)

    # M
    median_ARI_M.append(np.median(np.abs(ARI_lst)))
    mean_ARI_M.append(np.mean(np.abs(ARI_lst)))
    std_ARI_M.append(np.std(np.abs(ARI_lst)))

    median_ARI_M_1.append(np.median(np.abs(ARI_lst1)))
    mean_ARI_M_1.append(np.mean(np.abs(ARI_lst1)))
    std_ARI_M_1.append(np.std(np.abs(ARI_lst1)))

    median_ARI_M_2.append(np.median(np.abs(ARI_lst2)))
    mean_ARI_M_2.append(np.mean(np.abs(ARI_lst2)))
    std_ARI_M_2.append(np.std(np.abs(ARI_lst2)))

    median_ARI_M_3.append(np.median(np.abs(ARI_lst3)))
    mean_ARI_M_3.append(np.mean(np.abs(ARI_lst3)))
    std_ARI_M_3.append(np.std(np.abs(ARI_lst3)))

  return num_nodes, run_time, median_ARI_M_1, median_ARI_M_2, median_ARI_M_3, mean_ARI_M_1, mean_ARI_M_2, mean_ARI_M_3, std_ARI_M_1, std_ARI_M_2, std_ARI_M_3, num_evs, median_ARI_M, mean_ARI_M, std_ARI_M, ARI_lsts

et = time.time()
total_time = et - st
print("Execution time: ", total_time, " seconds")

m = 25
num_nodes, run_time, median_ARI_M_1, median_ARI_M_2, median_ARI_M_3, mean_ARI_M_1, mean_ARI_M_2, mean_ARI_M_3, std_ARI_M_1, std_ARI_M_2, std_ARI_M_3, num_evs, median_ARI_M, mean_ARI_M, std_ARI_M, ARI_lsts = GM_sys_M2_mixed_CC(n_lst, m, num_coords)